In [15]:
#install cupy, install sigpy, install pytorch
!pip install cupy-cuda113 #this may need to be updated depending on GPU/cuda
!pip install torch
!pip install numpy
!pip install sigpy
!pip install imageio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 7.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 15.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
import cupy
import sigpy
import torch
import numpy as np
from matplotlib import pyplot as plt
import imageio
import logging
from data_loader_utils import load_data
from CGs_layer import CG_DClayer
from gridded_calcs import init_im
from block_wise import Net_blockwise,SS_update,runner

In [2]:
#load temporally sorted k-space data and associated trajectories, density compensation and sense maps
ksp,coord,dcf,mpsa=load_data('MRI_Raw.h5',1)

test
1
coord
coils
0
1
2
3
4
5
6
7
8
9
10
11
12
noisy
stack
complete
0
1
2
3
4
5
6
7
8
9
10
11
12
[176, 176, 352]


JsenseRecon:   0%|          | 0/10 [00:00<?, ?it/s]

MaxEig:   0%|          | 0/500 [00:00<?, ?it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
0.4827706813812256


In [3]:

net_block=Net_blockwise(channels=2)
net_block=net_block.cuda()
optimizer0 = torch.optim.Adam(
    net_block.parameters(),
    lr= 1e-3)
alpha=torch.nn.parameter.Parameter(torch.ones([1],device='cuda')*.0002)

optimizer1=torch.optim.Adam([alpha],lr=1e-4)

In [6]:
def NN_unroll(im,kdelta,cdelta,ddelta,mps,alpha):
    lo=0
    im=net_block(im,lo,mps)
    im=CG_DClayer(im,kdelta,cdelta,ddelta,mps,alpha,3).unsqueeze(0)
    
    
    return im

In [ ]:
mps=torch.from_numpy(mpsa).cpu()
index_max=1
index_frame=1
for i in range(1000):
   
  #  torch.save(nets,'nets1')
    frac=.4
    ima,kgamma,dgamma,cgamma,kdelta,ddelta,cdelta=runner(frac,ksp,dcf,coord,mps.detach().cpu().numpy())
    alpha0=torch.zeros([1]).cuda()
    with torch.no_grad():
        im=CG_DClayer(ima.detach(),kdelta,cdelta,ddelta,mps,alpha0,5)
    im.requires_grad=True
    for j in range(5):
       
        im=torch.utils.checkpoint.checkpoint(NN_unroll,im,kdelta,cdelta,ddelta,mps,alpha)
       # im=im*.5+im_input*.5
        torch.cuda.empty_cache()
        import gc
        gc.collect()
    loss=SS_update(im,kgamma,dgamma,cgamma,mps)*10
    print(loss)
   # print(alpha)
    import imageio
    import imageio
    imageio.mimsave('image0.gif', [np.abs(im[i,30:120,110,30:220].detach().cpu().numpy())*1e15 for i in range(1)], fps=4)
 
    loss.backward()
    print(loss)
    optimizer0.step()
    optimizer0.zero_grad()
    optimizer1.step()
    optimizer1.zero_grad()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


tensor(11.4275, device='cuda:0', grad_fn=<MulBackward0>)
Parameter containing:
tensor([0.0002], device='cuda:0', requires_grad=True)
tensor(11.4275, device='cuda:0', grad_fn=<MulBackward0>)


tensor(10.9458, device='cuda:0', grad_fn=<MulBackward0>)
Parameter containing:
tensor([0.0003], device='cuda:0', requires_grad=True)
tensor(10.9458, device='cuda:0', grad_fn=<MulBackward0>)


tensor(11.4109, device='cuda:0', grad_fn=<MulBackward0>)
Parameter containing:
tensor([0.0004], device='cuda:0', requires_grad=True)
tensor(11.4109, device='cuda:0', grad_fn=<MulBackward0>)


tensor(11.1619, device='cuda:0', grad_fn=<MulBackward0>)
Parameter containing:
tensor([0.0005], device='cuda:0', requires_grad=True)
tensor(11.1619, device='cuda:0', grad_fn=<MulBackward0>)


...

tensor(10.8409, device='cuda:0', grad_fn=<MulBackward0>)
Parameter containing:
tensor([0.0005], device='cuda:0', requires_grad=True)
tensor(10.8409, device='cuda:0', grad_fn=<MulBackward0>)


tensor(12.0154, device='cuda:0', grad_fn=<MulBackward0>)
Parameter containing:
tensor([0.0006], device='cuda:0', requires_grad=True)
tensor(12.0154, device='cuda:0', grad_fn=<MulBackward0>)


tensor(14.6014, device='cuda:0', grad_fn=<MulBackward0>)
Parameter containing:
tensor([0.0006], device='cuda:0', requires_grad=True)
tensor(14.6014, device='cuda:0', grad_fn=<MulBackward0>)


tensor(11.7669, device='cuda:0', grad_fn=<MulBackward0>)
Parameter containing:
tensor([0.0006], device='cuda:0', requires_grad=True)
tensor(11.7669, device='cuda:0', grad_fn=<MulBackward0>)


tensor(10.6872, device='cuda:0', grad_fn=<MulBackward0>)
Parameter containing:
tensor([0.0006], device='cuda:0', requires_grad=True)
